In [1]:
import pandas as pd
import numpy as np

In [6]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')

In [3]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 90 or orientation_z < -90:
                motion_type = "Vertical Movement"
            elif orientation_x > 90 or orientation_x < -90:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [11]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Vertical Movement":
        predicted_activity = ["Running", "Walking", "Walking & party", "Football"]
    elif motion_type == "Horizontal Movement":
        predicted_activity = ["In vehicle", "Train", "In Bus", "On Bus stop"]
    elif motion_type == "Moving":
        predicted_activity = ["Cooking", "Meeting", "Movie", "Shopping & wearing"]
    elif motion_type == "Turning/Tilting":
        predicted_activity = ["Phone was out of the pocket (forgot)", "Took off glasses"]
    elif motion_type == "Stationary":
        predicted_activity = ["At home", "Pause", "Shop", "Work", "Eating", "Sleeping", "Watching TV", "Eating", "Picnic", "Video Games"]
    return predicted_activity

# Assuming 'pivot_data4' is your DataFrame
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [12]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,predicted_activity
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving,"[Cooking, Meeting, Movie, Shopping & wearing]"
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,"[Phone was out of the pocket (forgot), Took of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."


In [7]:
activity_types = activity_type_full['activity_type'].unique()

In [8]:
activity_types

array(['Video games', 'In computer', 'At home', 'Eat', 'Sleep',
       'Watching TV', 'In vehicle', 'Picnic ', 'Walk', 'Running',
       'Phone was out of the pocket (forgot)', 'Train', 'Cooking',
       'In bus', 'Meeting', 'Pause', 'Movie', 'Football', 'Shop',
       'Shopping& wearing', 'Walking&party', 'On bus stop',
       'Took off glasses', 'Work'], dtype=object)

In [4]:
pivot_data4_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv'

pivot_data4.to_csv(pivot_data4_pathway, index=False)

In [5]:
pivot_data4['motion_type'].value_counts()


motion_type
Stationary             156088
Turning/Tilting          8986
Moving                   7795
Vertical Movement         502
Horizontal Movement       291
Name: count, dtype: int64

This diversity of motion_types shows that the function is working. 

In [10]:
import pandas as pd
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch

# Initialize Potassium app
app = Potassium("my_app")

# Initialize BERT model
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Define the BERT model using the pipeline interface from transformers
    bert_model = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    context = {
        "model_b": bert_model,
    }

    return context

# Handler to process each row and append predicted activity to pivot_data4
def process_rows(context: dict, pivot_data4: pd.DataFrame) -> pd.DataFrame:
    # Retrieve the BERT model from the context
    model = context.get("model_b")

    # Initialize empty lists to store predicted activities and substitutions
    predicted_activities = []
    substitutions = []

    # Iterate over each row of pivot_data4
    for index, row in pivot_data4.iterrows():
        # Extract motion type from the current row
        motion_type = row['motion_type']

        # Create prompt using the motion type and [MASK] token
        prompt = f"I am {motion_type} thus am performing [MASK]"

        # Generate predictions using the BERT model
        predictions = model(prompt)

        # Extract the first prediction suggestion for the [MASK] token
        suggestion = predictions[0]['token_str']

        # Append the predicted activity and suggestion to the respective lists
        predicted_activities.append(predictions[0]['sequence'])
        substitutions.append(suggestion)

    # Add the lists of predicted activities and suggestions as new columns to pivot_data4
    pivot_data4['predicted_activity'] = predicted_activities
    pivot_data4['suggestion_for_mask'] = substitutions

    return pivot_data4

# Endpoint to trigger the processing of rows
@app.handler("/process_rows")
def handler(context: dict, request: Request) -> Response:
    # Retrieve pivot_data4 from the request payload
    pivot_data4 = request.json.get("pivot_data4")

    # Process rows and append predicted activities and substitutions
    pivot_data4_with_predictions = process_rows(context, pd.DataFrame(pivot_data4))

    # Return the updated pivot_data4 with predicted activities and substitutions
    return Response(
        json=pivot_data4_with_predictions.to_dict(orient='records'),
        status=200
    )

# Run the Potassium app
if __name__ == "__main__":
    app.serve()


------
Starting Potassium Server 🍌
Running init()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Started 1 workers
Serving at http://0.0.0.0:8000
------


In [7]:
import pandas as pd
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch

# Initialize Potassium app
app = Potassium("my_app")

# Initialize BERT model
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Define the BERT model using the pipeline interface from transformers
    bert_model = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    context = {
        "model_b": bert_model,
    }

    return context

# Handler to process each row and append predicted activity to pivot_data4
def process_rows(context: dict, pivot_data4: pd.DataFrame) -> pd.DataFrame:
    # Retrieve the BERT model from the context
    model = context.get("model_b")

    # Iterate over each row of pivot_data4
    for index, row in pivot_data4.iterrows():
        # Extract motion type from the current row
        motion_type = row['motion_type']

        # Create prompt using the motion type and [MASK] token
        prompt = f"I am {motion_type} thus am performing [MASK]"

        # Generate predictions using the BERT model
        predictions = model(prompt)

        # Extract the suggested word from BERT's predictions for [MASK]
        suggested_word = predictions[0]['token_str']

        # Append the suggested word to the predicted_activity column
        pivot_data4.at[index, 'predicted_activity'] = suggested_word

    return pivot_data4

# Endpoint to trigger the processing of rows
@app.handler("/process_rows")
def handler(context: dict, request: Request) -> Response:
    # Retrieve pivot_data4 from the request payload
    pivot_data4 = request.json.get("pivot_data4")

    # Process rows and append predicted activities
    pivot_data4_with_predictions = process_rows(context, pd.DataFrame(pivot_data4))

    # Return the updated pivot_data4 with predicted activities
    return Response(
        json=pivot_data4_with_predictions.to_dict(orient='records'),
        status=200
    )

# Run the Potassium app
if __name__ == "__main__":
    app.serve()

------
Starting Potassium Server 🍌
Running init()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Started 1 workers
Serving at http://0.0.0.0:8000
------


In [8]:
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch
import time

app = Potassium("my_app")

# @app.init runs at startup, and initializes the app's context
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Defind the model, using the "pipeline" interface from transformers
    bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    
    context = {
        "model_b": bertModel,
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/bert")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    model = context.get("model_b")

    # our prompt is from the 'prompt' key in the user request
    prompt = request.json.get("prompt")

    # the outputs is created by passing the user request to the model
    outputs = model(prompt)

    # returned
    return Response(
        json = {"outputs": outputs}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

KeyboardInterrupt: 

In [2]:
from potassium import Potassium, Request, Response
# from transformers import pipeline
import torch

app = Potassium("my_app")

def custom_model(inputData):
  outcome = "Hello from a function, this is your input data: " + inputData
  print(outcome) 
  return outcome

# @app.init runs at startup, and initializes the app's context
@app.init
def init():

    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Define the model, using the "pipeline" interface from transformers
    # bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)

    # define any model that you understand how to process
    # model = custom_model
    
    context = {
        # "model_b": bertModel,
        # model: model
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/custom")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    # model = context.get("model")
    # model = custom_model

    # our prompt is from the 'prompt' key in the user request
    # input = request.json.get("prompt")

    inputData = 'this is where my input data will be.'

    # the outputs is created by passing the user request to the model
    outcome = custom_model(inputData)

    print(outcome)

    # returned
    return Response(
        json = {"outputs": outcome}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

------
Starting Potassium Server 🍌
Running init()
Started 1 workers
Serving at http://0.0.0.0:8000
------
127.0.0.1 - - [22/Mar/2024 09:05:44] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [22/Mar/2024 09:05:45] "GET /favicon.ico HTTP/1.1" 405 -
127.0.0.1 - - [22/Mar/2024 09:05:58] "GET / HTTP/1.1" 405 -
